<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/MPNN_CNN_%26_DeepDTI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
################################
#This script provide a demo of MPNN_CNN & DeepDTI, the runtime on one fold mainly takes 3~5 hours (V100).

In [13]:
import torch

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
device

device(type='cpu')

In [16]:
pip install DeepPurpose

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolu

In [ ]:
pip install rdkit

In [ ]:
pip install git+https://github.com/bp-kelley/descriptastorus

In [ ]:
pip install pandas-flavor

In [ ]:
from DeepPurpose import utils, dataset
from DeepPurpose import DTI as models
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from sklearn import metrics
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:

#Load Data
################################################################


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
#The header=None argument tells pandas not to treat the first row of the file as a header.
dt_08.columns = ['head','relation','tail']
#This line renames the columns of the dt_08 DataFrame to ['head','relation','tail'].

df_drug = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')
#the data in a pandas DataFrame

df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']
#This line renames the columns of the df_proseq DataFrame to ['pro_id','pro_ids','seq']

pro_id = df_proseq['pro_id']
#This line extracts the 'pro_id' column from the df_proseq DataFrame and stores it in the pro_id variable.

In [ ]:
# print(df_drug)

In [ ]:
# print(df_proseq)

In [ ]:

#define function
################################


In [ ]:
def get_structure(data, df_drug, df_proseq):
    drug_struc = pd.merge(data, df_drug, how='left', left_on='head', right_on='drug_id')['smiles'].values
    pro_struc = pd.merge(data, df_proseq, how='left', left_on='tail', right_on='pro_id')['seq'].values

    return pd.DataFrame({'label': data['label'].values, 'drug_struc': drug_struc, 'pro_struc': pro_struc})

In [ ]:
# def get_struc(data,df_drug,df_proseq):
#   #This line defines a function named get_struc that takes three arguments: data, df_drug, and df_proseq. These arguments represent a pandas DataFrame containing data about molecules and their labels,
#   # a pandas DataFrame containing drug information,
#   #and a pandas DataFrame containing protein sequence information, respectively.

#     drug_struc = pd.merge(data,df_drug,how='left',left_on='head',right_on='drug_id')['smiles'].values
# #This line performs a left merge of data and df_drug DataFrames using 'head' as the key column from data and 'drug_id' as the key column from df_drug.
# # The result of this merge operation is a new DataFrame where rows from both DataFrames are combined based on the common key values.
# #Then, this line extracts the 'smiles' column from the resulting DataFrame and converts it into a numpy array.
# # This numpy array contains the SMILES (Simplified Molecular Input Line Entry System) representations of the drug molecules.

#     pro_struc = pd.merge(data,df_proseq,how='left',left_on='tail',right_on='pro_id')['seq'].values
# #This line performs a similar left merge operation,
# # this time merging data and df_proseq DataFrames using 'tail' as the key column from data and 'pro_id' as the key column from df_proseq.
# #The result of this merge operation is a new DataFrame where rows from both DataFrames are combined based on the common key values.
# # Then, this line extracts the 'seq' column from the resulting DataFrame and converts it into a numpy array.
# #This numpy array contains the amino acid sequences of the protein sequences.

#     return pd.DataFrame(data['label'].values,drug_struc,pro_struc)
# #Finally, this line returns a tuple containing three numpy arrays.
# #The first array contains the labels of the molecules.
# # The second array contains the SMILES representations of the drug molecules.
# # The third array contains the amino acid sequences of the protein sequences.
# ####Overall, the get_struc function serves the purpose of retrieving the structure information of molecules and protein sequences from the provided DataFrames and returning these structure information as numpy arrays.

# # من درا
#     return pd.DataFrame(drug_struc,pro_struc)



def roc_auc(y,pred):
    fpr, tpr, _ = metrics.roc_curve(y, pred)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc
###Sources: w3schools.com (1) analyticsvidhya.com (2) scikit-learn.org (3) stackoverflow.com (4)
###The roc_auc_score function in Python, provided by the sklearn.metrics module,
# computes the area under the receiver operating characteristic (ROC) curve.
# The ROC curve is a graphical representation of the performance of a binary classifier system as it varies its decision threshold.
# The roc_auc_score function takes as input two arrays:
#actual: Ground truth (true) labels for the test set, can be either a binary array or a multilabel array.
#predicted: Predicted labels for the test set, can be either a binary array or a continuous array representing prediction scores.
#If predicted is passed as a binary array, the function considers the entries as "scores" rather than probabilities; therefore,
#the results may match accuracy values, depending on the threshold used for binarization.
# To obtain meaningful results, it's recommended to pass a continuous array containing prediction scores for each sample.



def pr_auc(y, pred):
  #Within the function, we call the precision_recall_curve method from the sklearn.metrics module.
  # This method computes precision-recall pairs for different probability thresholds.
  #The arguments passed to this method are y, the true labels, and pred, the predicted probabilities.
    precision, recall, _ = metrics.precision_recall_curve(y, pred)
#After computing the precision-recall pairs,
#we compute the area under the precision-recall curve (AUPRC) using the auc method from the sklearn.metrics module.
#The arguments passed to this method are recall, the recall values computed earlier, and precision, the precision values computed earlier.
    pr_auc = metrics.auc(recall, precision)
#Finally, the function returns the computed AUPRC
    return pr_auc


data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'
def load_data(i):
  #This line defines a function named load_data that takes a single argument, i.
  # This function will load the train and test datasets for the i-th fold in the cross-validation process.
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')
#nside the function, we use the read_csv function from the pandas library to read the train data for the i-th fold from a csv file.
#The file path is constructed by concatenating the data_path variable with the file name.
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')
#Similar to the line above, we use the read_csv function to read the test data for the i-th fold from a csv file.
    return train,test
    #Finally, the function returns the train and test datasets as a tuple.

In [ ]:
# train,test=load_data(9)

In [ ]:
# train_label = train['label']
# test_label = test['label']
#     #train_label = train_all['label']: Here, we extract the 'label' column from the train_all DataFrame and store it in the variable train_label.
#     #test_label = test_all['label']: Similar to the previous line, we extract the 'label' column from the test_all DataFrame and store it in the variable test_label.


# train_re, valid_re, y_train, y_valid = train_test_split(train[['head','relation','tail']],train_label,test_size=0.01,
#                                                                 random_state=0,
#                                                                 stratify=train_label)

In [ ]:
# print(train_re)

In [ ]:
# # Extract the label column from the train DataFrame
# label_column = train['label']

# # Add the label column to the train_re DataFrame
# train_re = train_re.assign(label=label_column)
# # train_re = train_re.drop('relation', axis=1)

In [ ]:
# print(train_re)

In [ ]:
# # train_drug_feats,train_pro_feats = get_struc(train_re,df_drug,df_proseq)
# train_drug_feats, train_pro_feats = get_structure(train_re, df_drug, df_proseq)

In [ ]:
# # Assuming the get_structure() function is defined elsewhere in your code
# train_re_with_features = get_structure(train_re, df_drug, df_proseq)

# # Now you can access the drug and protein structures from the train_re_with_features DataFrame
# train_drug_feats = train_re_with_features['drug_struc'].values
# train_pro_feats = train_re_with_features['pro_struc'].values

In [ ]:
# print(train)

In [ ]:
# train[1]["label"]

در این کد از تابع get_input برای پیش پردازش داده های ورودی استفاده می شود. تابع دو شی DataFrame، train_all و test_all را به عنوان آرگومان می گیرد.

این تابع با استخراج ستون 'برچسب' از هر دو DataFrame و ذخیره آنها در متغیرهای جداگانه شروع می شود.

مرحله بعدی تقسیم داده های آموزشی به مجموعه های آموزشی و اعتبار سنجی است. برای این منظور از تابع train_test_split از کتابخانه scikit-learn استفاده می شود. داده های آموزشی به 99% (مجموعه های آموزش و اعتبارسنجی) و 1% (مجموعه تست) تقسیم می شوند. همچنین داده ها بر اساس مقادیر برچسب تقسیم می شوند تا اطمینان حاصل شود که تقسیم توزیع کلاس را حفظ می کند.

پس از تقسیم داده های آموزشی، تابع get_struc فراخوانی می شود تا ویژگی های توالی دارو و پروتئین را برای هر سه در مجموعه داده استخراج کند.

سپس تابع utils.data_process برای پیش پردازش داده ها فراخوانی می شود. این تابع وظایفی مانند محاسبه اثر انگشت مولکولی برای مولکول های دارو، تولید بردارهای ورودی لازم برای لایه های MPNN و CNN و غیره را بر عهده دارد.

سپس داده های آموزش پردازش شده، اعتبارسنجی و آزمون توسط تابع برگردانده می شود.

مراحل پیش پردازش داده ها، از جمله تقسیم داده ها به مجموعه های آموزشی و اعتبارسنجی، محاسبه اثر انگشت مولکولی، و تولید بردارهای ورودی، برای عملکرد صحیح مدل های MPNN و CNN ضروری هستند. مدل MPNN از شبکه‌های عصبی نموداری برای یادگیری نمایش‌های توالی دارو و پروتئین استفاده می‌کند، در حالی که مدل CNN از شبکه‌های عصبی کانولوشن برای یادگیری نمایش‌های دنباله هدف استفاده می‌کند. هر دو مدل به داده های ورودی در قالب خاصی نیاز دارند و این قالب توسط تابع utils.data_process تولید می شود.</s

In [ ]:

#need to be adjusted when changing methods
drug_encoding, target_encoding = 'MPNN', 'CNN'


در اینجا، مقادیر رشته "MPNN" و "CNN" را به ترتیب به متغیرهای drug_encoding و target_encoding اختصاص می دهیم. این متغیرها نشان دهنده روش های رمزگذاری مورد استفاده برای مولکول های دارو و توالی پروتئین در مدل های MPNN و CNN هستند.

In [ ]:
def get_input(train,test):
  #این خط تابع get_input را تعریف می کند که دو شی DataFrame را به عنوان آرگومان ورودی می گیرد.
  #انتظار می رود این DataFrames حاوی داده های سه گانه و برچسب از پیش پردازش شده برای مجموعه های آموزشی و آزمایشی باشد.
    train_label = train['label']
    test_label = test['label']
    #train_label = train_all['label']: Here, we extract the 'label' column from the train_all DataFrame and store it in the variable train_label.
    #test_label = test_all['label']: Similar to the previous line, we extract the 'label' column from the test_all DataFrame and store it in the variable test_label.


    train_re, valid_re, y_train, y_valid = train_test_split(train[['head','relation','tail']],train_label,test_size=0.01,
                                                                random_state=0,
                                                                stratify=train_label)
    #this line splits the training data into training and validation sets.
    # The train_test_split function from the scikit-learn library is used for this purpose.
    #The train_size argument is set to 0.99, meaning that the training set will contain 99% of the data.
    # The random_state argument is set to 0, ensuring that the same split is produced each time the code is run.
    # The stratify argument is set to train_label, meaning that the split will maintain the class distribution of the labels in the training set.


    # Extract the label column from the train DataFrame
    label_column = train['label']

# Add the label column to the train_re DataFrame
    train_re = train_re.assign(label=label_column)
    valid_re = train_re.assign(label=label_column)

    # Assuming the get_structure() function is defined elsewhere in your code
    train_re_with_features = get_structure(train_re, df_drug, df_proseq)

    # Now you can access the drug and protein structures from the train_re_with_features DataFrame
    train_drug_feats = train_re_with_features['drug_struc'].values
    train_pro_feats = train_re_with_features['pro_struc'].values
#this line calls the get_struc function to extract the drug and protein sequence features for each triple in the training set.
#The get_struc function is expected to return two lists or arrays: train_drug_feats and train_pro_feats.

#     valid_drug_feats,valid_pro_feats = get_struc(valid_re,df_drug,df_proseq)
# # Similar to the previous line
#     test_drug_feats,test_pro_feats = get_struc(test,df_drug,df_proseq)
# # Similar to the previous line
        # Assuming the get_structure() function is defined elsewhere in your code
    valid_re_with_features = get_structure(valid_re, df_drug, df_proseq)

    # Now you can access the drug and protein structures from the train_re_with_features DataFrame
    valid_drug_feats = train_re_with_features['drug_struc'].values
    valid_pro_feats = train_re_with_features['pro_struc'].values

    test_with_features = get_structure(test, df_drug, df_proseq)

    # Now you can access the drug and protein structures from the train_re_with_features DataFrame
    test_drug_feats = train_re_with_features['drug_struc'].values
    test_pro_feats = train_re_with_features['pro_struc'].values














    train = utils.data_process(train_drug_feats, train_pro_feats, y_train,
                                drug_encoding, target_encoding,
                                split_method='no_split',
                                random_seed = 0)
#This line calls the utils.data_process function to preprocess the training data.
#The data_process function is expected to return a preprocessed dataset object that can be used as input to the MPNN or CNN model.
    valid = utils.data_process(valid_drug_feats, valid_pro_feats, y_valid,
                            drug_encoding, target_encoding,
                            split_method='no_split',
                            random_seed = 0)
# Similar to the previous line
    test = utils.data_process(test_drug_feats, test_pro_feats, test_label,
                                drug_encoding, target_encoding,
                                split_method='no_split',
                                random_seed = 0)
# Similar to the previous line
    return train,valid,test



In [ ]:

######################################## Training
#parameters for MPNN_CNN
config = utils.generate_config(
                        drug_encoding = drug_encoding,
                        target_encoding = target_encoding,
                        cls_hidden_dims = [1024,1024,512],
                        train_epoch = 2,
                        LR = 0.001,
                        batch_size = 5000,
                        hidden_dim_drug = 16,
                        mpnn_hidden_size = 16,
                        mpnn_depth = 3,
                        cnn_target_filters = [32,64,64],
                        cnn_target_kernels = [4,8,8]
                        )
#The generate_config function is a utility function that creates a configuration object for our MPNN-CNN model. It takes the following arguments:
# drug_encoding: A function that converts the drug input into an appropriate encoding.
# target_encoding: A function that converts the target input into an appropriate encoding.
# cls_hidden_dims: A list of integers representing the hidden dimensions of the classifier.
# train_epoch: The number of training epochs.
# LR: The learning rate for the model.
# batch_size: The number of samples to include in each training batch.
# hidden_dim_drug: The dimension of the drug representation in the MPNN-CNN model.
# mpnn_hidden_size: The dimension of the MPNN representation.
# mpnn_depth: The depth of the MPNN representation.
# cnn_target_filters: A list of integers representing the number of filters for each convolutional layer in the CNN target representation.
# cnn_target_kernels: A list of integers representing the kernel size for each convolutional layer in the CNN target representation.

In [ ]:
for i in range(10):
    print(i)
    train,test = load_data(i)
    # train=pd.DataFrame(train)
    # train=pd.DataFrame(test)
    # print(train["label"])
    # print(test["label"])
    train_input,valid_input,test_input = get_input(train,test)

    model = models.model_initialize(**config)
    # This line creates a new machine learning model.
    # models.model_initialize is a function that takes a dictionary of configuration parameters and returns a model.
    model.train(train_input,valid_input)
    #Finally, we train the model on the train_input data, using the valid_input data for validation.
    #This step involves adjusting the model's parameters to minimize the error between its predictions and the actual data.


In [ ]:
print(type(train))

The command model = models.model_initialize(**config) initializes a new machine learning model, using the specified configuration parameters. The specific type of model that this command creates depends on the config dictionary that you pass to it.

For example, if you want to create a logistic regression model, you might pass a config dictionary like this:

python
Download
Copy code
config = {
    'model_type': 'logistic_regression',
    'regularization': 0.1,
    'learning_rate': 0.01,
    'epochs': 100
}

model = models.model_initialize(**config)
In this case, the model_initialize function will create a new logistic regression model, with the specified regularization parameter, learning rate, and number of epochs.

Similarly, if you want to create a neural network model, you might pass a config dictionary like this:

python
Download
Copy code
config = {
    'model_type': 'neural_network',
    'layers': [32, 16, 8],
    'activation': 'relu',
    'optimizer': 'adam',
    'learning_rate': 0.01,
    'epochs': 100
}

model = models.model_initialize(**config)
In this case, the model_initialize function will create a new neural network model, with the specified architecture (layers), activation function, optimizer, learning rate, and number of epochs.

In general, the model_initialize function uses the config dictionary to determine which type of model to create, and what parameters to use when initializing that model.

Remember, the actual implementation of the model_initialize function will depend on the specific machine learning library that you are using. However, the basic concept of using a configuration dictionary to specify the model type and parameters is the same across most machine learning libraries.





برای چک کردن


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10/test_fold_1.csv")

In [ ]:
train["label"]

In [ ]:
data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'
def load_data(i):
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')
    return train,test

In [ ]:
train , test=load_data(1)

In [ ]:
test["label"]

In [ ]:
get_input(train,test)